<a href="https://colab.research.google.com/github/YashviShahBvn/AI-Mini-Projects/blob/main/CV_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# All libraries
import cv2
import numpy as np
from pathlib import Path

In [3]:
# Main class

class ObjectDetector:
    def __init__(self):
        self.net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg") # Pretrained weights
        self.classes = []
        with open("coco.names", "r") as f: # names of the classes
            self.classes = [line.strip() for line in f.readlines()]

        self.layer_names = self.net.getLayerNames()
        self.output_layers = [self.layer_names[i - 1]
                             for i in self.net.getUnconnectedOutLayers()]
        self.colors = np.random.uniform(0, 255, size=(len(self.classes), 3))

    def detect_objects(self, image_path):
        img = cv2.imread(image_path)
        height, width, channels = img.shape

        blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        self.net.setInput(blob)
        outs = self.net.forward(self.output_layers)

        class_ids = []
        confidences = []
        boxes = []

        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(self.classes[class_ids[i]])
                color = self.colors[class_ids[i]]
                cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
                cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        output_path = 'detected_' + Path(image_path).name
        cv2.imwrite(output_path, img)
        print(f"Detection complete. Saved to {output_path}")
        return class_ids, confidences, boxes

## Explanation

### Constructor

1. Load weights and structure (cgf)
2. Load classes names from coco dataset (80 classes)
3. Get Netwrok Layers => YOLOv3 has 3 output layers
    1. 13x13 grid
    2. 26x26 grid
    3. 52x52 grid
4. Generate colours for visualization

### Detect object (main function)

1. Load and preprocess image (create BLOB, resize, convert BGR→RGB)
2. Feed image to network and get output detections in "outs" var => This var will contain of the 3 types of output provided by YOLOv3
3. Process all raw detections
    1. Filter based on confidence
    2. NMS => Remove overlapping boxes, etc
4. Draw outlines on boxes
5. Save final image

In [4]:
# testing
if __name__ == "__main__":
    detector = ObjectDetector()
    detector.detect_objects('Yolo_Test_image.jpg')

Detection complete. Saved to detected_Yolo_Test_image.jpg
